<a href="https://colab.research.google.com/github/sagnikmukherjee660-extc/CNN_CIFAR_Classifier-/blob/main/CNN_CIFAR10_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 5.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, utils
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import keras_tuner as kt

Load & preprocess CIFAR‑10

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32') / 255.0
y_train = utils.to_categorical(y_train, 10)
y_test  = utils.to_categorical(y_test, 10)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


MixUp data generator

In [ ]:
def mixup_generator(x, y, batch_size=128, alpha=0.2):
    n = x.shape[0]
    idx = np.arange(n)
    while True:
        np.random.shuffle(idx)
        for i in range(0, n, batch_size):
            batch_idx = idx[i : i + batch_size]
            x1, y1 = x[batch_idx].copy(), y[batch_idx].copy()
            lam = np.random.beta(alpha, alpha, size=len(x1))
            lam_x = lam.reshape(-1,1,1,1)
            lam_y = lam.reshape(-1,1)
            idx2 = np.random.choice(n, size=len(x1), replace=False)
            x2, y2 = x[idx2], y[idx2]
            x_batch = lam_x * x1 + (1 - lam_x) * x2
            y_batch = lam_y * y1 + (1 - lam_y) * y2
            yield x_batch, y_batch

Build tunable model using ResNet50 backbone

In [ ]:
def build_model(hp):
    # Backbone
    base = tf.keras.applications.ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(32,32,3),
        pooling='avg'
    )
    base.trainable = False

    inputs = layers.Input(shape=(32,32,3))
    x = tf.keras.applications.resnet.preprocess_input(inputs)
    x = base(x, training=False)

    # Tunable dense layer size
    units = hp.Choice('dense_units', [128, 256, 512], default=256)
    x = layers.Dense(units, activation='relu')(x)
    x = layers.Dropout(
        hp.Float('dropout_rate', min_value=0.3, max_value=0.6, step=0.1, default=0.5)
    )(x)

    outputs = layers.Dense(10, activation='softmax')(x)
    model = models.Model(inputs, outputs)

    # Compile with tunable learning rate and label smoothing
    lr = hp.Float('learning_rate', 1e-4, 1e-2, sampling='log', default=1e-3)
    loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss=loss,
        metrics=['accuracy']
    )
    return model

Set up Keras Tuner (Hyperband)

In [ ]:
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    factor=3,
    directory='cifar5_tuner',
    project_name='episode5'
)

# Early stopping during tuning
stop_early = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Run hyperparameter search

In [ ]:
batch_size = 128
steps_per_epoch = x_train.shape[0] // batch_size

tuner.search(
    mixup_generator(x_train, y_train, batch_size=batch_size, alpha=0.2),
    steps_per_epoch=steps_per_epoch,
    validation_data=(x_test, y_test),
    epochs=20,
    callbacks=[stop_early]
)

Epoch 1/3
390/390 ━━━━━━━━━━━━━━━━━━━━ 238s 578ms/step - accuracy: 0.1434 - loss: 2.3849 - val_accuracy: 0.2553 - val_loss: 2.1395
Epoch 2/3
110/390 ━━━━━━━━━━━━━━━━━━━━ 2:04 445ms/step - accuracy: 0.2085 - loss: 2.1884

Retrieve the best model

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]

Fine Tune

In [ ]:
fine_tune_history = best_model.fit(
    mixup_generator(x_train, y_train, batch_size=batch_size, alpha=0.2),
    steps_per_epoch=steps_per_epoch,
    validation_data=(x_test, y_test),
    epochs=10,
    callbacks=[
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
        EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, verbose=1)
    ]
)

Final evaluation

In [ ]:
test_loss, test_acc = best_model.evaluate(x_test, y_test, verbose=2)
print(f"Final test accuracy: {test_acc:.4f}")